# data

In [2]:
%load_ext autoreload
%autoreload 2
import importlib
from importlib import reload  
  
import monotonic
import utils
from train import execute_model_evaluation
from train_config import FlatACLConfig
import train_config
import data_manager
from data_manager import DatasetManager
import metrics
from dataclasses import replace

reload(utils)
reload(monotonic)
reload(train_config)

# Initialize the data manager
data_mgr = DatasetManager(device='cuda')

# Load a synthetic dataset
data_mgr.load_trajectory('lorenz96', steps=18999, dt=1e-2) # 50399
# SCALE = False
# trajectory = utils.generate_trajectory('lorenz',steps=52200, dt=1e-2) 
# trajectory = utils.generate_hyperchaotic_rossler(steps=12000, dt=1e-3)
# trajectory_2 = utils.generate_henon(steps=52000) 

Lorenz96System initialized with method: rk4 on device: cuda

Dataset: lorenz96 (synthetic)
Shape: torch.Size([19000, 6])
Channels: 6
Length: 19000
Parameters: {'steps': 18999, 'dt': 0.01}

Sample data (first 2 rows):
tensor([[8.0100, 8.0000, 8.0000, 8.0000, 8.0000, 8.0000],
        [8.0099, 7.9999, 7.9992, 8.0000, 8.0001, 8.0008]], device='cuda:0')


# Lorenz96 - forcing 8.0

## Seq=96

### EigenACL

#### pred=96

In [ ]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

##### do not use: (8,4) rotations

In [ ]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
Train set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 103
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 96, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 103
Validation Batches: 14
Test Batches: 29

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 9.3481, mae: 2.3189, huber: 1.8744, swd: 3.9977, target_std: 3.6910
Epoch [1/50], Val Losses: mse: 7.9674, mae: 2.1180, huber: 1.6820, swd: 2.4753, target_std: 3.8473
Epoch [1/50], Test Losses: mse: 7.9356, mae: 2.0

#### pred=196

In [ ]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

##### do not use: ablation: (8,4) rotations

In [61]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
Train set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 102
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 196, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 102
Validation Batches: 13
Test Batches: 28

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 11.1600, mae: 2.6193, huber: 2.1630, swd: 4.3351, target_std: 3.6898
Epoch [1/50], Val Losses: mse: 10.5463, mae: 2.5053, huber: 2.0542, swd: 3.0806, target_std: 3.8710
Epoch [1/50], Test Losses: mse: 10.5614, m

#### pred=336

##### huber

In [15]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 101
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 101
Validation Batches: 12
Test Batches: 27

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 12.4120, mae: 2.8243, huber: 2.3620, swd: 4.5900, ept: 51.7769
Epoch [1/50], V

##### do not use: ablation: (8,4) rotations

In [62]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
Train set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 101
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 101
Validation Batches: 12
Test Batches: 27

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 12.2774, mae: 2.7969, huber: 2.3357, swd: 4.5690, target_std: 3.6905
Epoch [1/50], Val Losses: mse: 11.7301, mae: 2.7035, huber: 2.2453, swd: 3.0335, target_std: 3.8842
Epoch [1/50], Test Losses: mse: 12.1381, m

#### pred=720

##### huber

In [ ]:
from monotonic import DynamicTanh
import torch.nn as nn

importlib.reload(monotonic)
importlib.reload(train_config) 
cfg = train_config.FlatACLConfig(  # original householder 
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128, 
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=True, 
    ablate_shift_inside_scale=False,
    householder_reflects_latent = 2,
    householder_reflects_data = 4,
    mixing_strategy='delay_only', 
    loss_backward_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    loss_validate_weights = [0.0, 0.0, 1.0, 0.0, 0.0],
    ablate_deterministic_y0=False, 
)
cfg.x_to_z_delay.enable_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_delay.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_delay.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_delay.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_delay.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.x_to_z_deri.enable_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_scale_shift = [True, False]
cfg.x_to_z_deri.spectral_flags_magnitudes = [False, True]
cfg.x_to_z_deri.spectral_flags_hidden_layers = [False, False]
cfg.x_to_z_deri.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.x_to_z_deri.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_x_main.enable_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_x_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_x_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_x_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_x_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_push_to_z.enable_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_scale_shift = [True, False]
cfg.z_push_to_z.spectral_flags_magnitudes = [False, True]
cfg.z_push_to_z.spectral_flags_hidden_layers = [False, False]
cfg.z_push_to_z.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_push_to_z.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]

cfg.z_to_y_main.enable_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_scale_shift = [True, False]
cfg.z_to_y_main.spectral_flags_magnitudes = [False, True]
cfg.z_to_y_main.spectral_flags_hidden_layers = [False, False]
cfg.z_to_y_main.activations_scale_shift = ["relu6", "dynamic_tanh"]
cfg.z_to_y_main.activations_hidden_layers = [nn.ELU, nn.LogSigmoid]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

##### do not use: ablation: (8,4) rotations

In [94]:
importlib.reload(monotonic)
importlib.reload(train_config)

cfg = train_config.FlatACLConfig( 
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['lorenz96']['channels'],# data_mgr.channels,              # ← number of features in your data
    batch_size=128,
    learning_rate=9e-4, 
    seeds=[1955, 7, 20],  
    epochs=50, 
    dim_hidden=128,
    dim_augment=128,
    ablate_no_koopman=False,
    use_complex_eigenvalues=True,
    second_delay_use_shift=True,
    ablate_rotate_back_Koopman=False, 
    ablate_shift_inside_scale=False,
)
cfg.x_to_z_delay.enable_magnitudes_scale_shift = [False, True]
cfg.x_to_z_deri.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_x_main.enable_magnitudes_scale_shift = [False, True]
cfg.z_to_y_main.enable_magnitudes_scale_shift = [False, True]
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
Train set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 98
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 720, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 98
Validation Batches: 9
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 13.1797, mae: 2.9450, huber: 2.4795, swd: 4.8740, target_std: 3.6914
Epoch [1/50], Val Losses: mse: 13.3269, mae: 2.9438, huber: 2.4791, swd: 3.9701, target_std: 3.9215
Epoch [1/50], Test Losses: mse: 13.4754, mae:

### Timemixer

#### pred=96

In [3]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp_mixer_96_96 = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 103
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 96, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 103
Validation Batches: 14
Test Batches: 29

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 15.1232, mae: 2.982

#### pred=196

In [4]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp_mixer_96_96 = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 102
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 196, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 102
Validation Batches: 13
Test Batches: 28

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 17.7753, mae: 3

#### pred=336

In [5]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp_mixer_96_96 = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 101
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 101
Validation Batches: 12
Test Batches: 27

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 20.1417, mae: 3

#### pred=720

In [6]:
utils.reload_modules([utils])
cfg = train_config.FlatTimeMixerConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp_mixer_96_96 = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)


Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 98
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 720, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 98
Validation Batches: 9
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 19.7920, mae: 3.57

### PatchTST

#### pred=96

In [7]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 103
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 96, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 103
Validation Batches: 14
Test Batches: 29

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 12.2987, mae: 2.616

#### pred=196

In [8]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 102
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 196, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 102
Validation Batches: 13
Test Batches: 28

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 14.4070, mae: 2

#### pred=336

In [9]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 101
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 101
Validation Batches: 12
Test Batches: 27

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.1327, mae: 3

#### pred=720

In [10]:
utils.reload_modules([utils])
cfg = train_config.FlatPatchTSTConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['lorenz96']['channels'],
    enc_in=data_mgr.datasets['lorenz96']['channels'],
    dec_in=data_mgr.datasets['lorenz96']['channels'],
    c_out=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50,
    task_name='long_term_forecast',
    factor=3,
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 98
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 720, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 98
Validation Batches: 9
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.1380, mae: 3.22

### DLinear

#### pred=96

In [11]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=96,
    channels=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([96, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 103
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 96, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 96
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 103
Validation Batches: 14
Test Batches: 29

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 14.2347, mae: 2.980

#### pred=196

In [12]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=196,
    channels=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([196, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 102
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 196, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 196
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 102
Validation Batches: 13
Test Batches: 28

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 15.3432, mae: 3

#### pred=336

In [13]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 101
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 101
Validation Batches: 12
Test Batches: 27

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.1786, mae: 3

In [66]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=336,
    pred_len=336,
    channels=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
Train set sample shapes: torch.Size([336, 6]), torch.Size([336, 6])
Validation set sample shapes: torch.Size([336, 6]), torch.Size([336, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 99
Batch 0: Data shape torch.Size([128, 336, 6]), Target shape torch.Size([128, 336, 6])

Data Preparation: lorenz96
Sequence Length: 336
Prediction Length: 336
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 99
Validation Batches: 10
Test Batches: 25

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 14.2220, mae: 3.0295, huber: 2.5647, swd: 4.7813, target_std: 3.6922
Epoch [1/50], Val Losses: mse: 14.9550, mae: 3.1237, huber: 2.6581, swd: 6.07

#### pred=720

In [14]:
importlib.reload(monotonic)
importlib.reload(train_config)
utils.reload_modules([utils])
cfg = train_config.FlatDLinearConfig(
    seq_len=96,
    pred_len=720,
    channels=data_mgr.datasets['lorenz96']['channels'],
    batch_size=128,
    learning_rate=9e-4,
    seeds=[1955, 7, 20],
    epochs=50, 
)
exp = execute_model_evaluation('lorenz96', cfg, data_mgr, scale=False)

Reloading modules...
  Reloaded: utils
Module reload complete.
Shape of training data: torch.Size([13300, 6])
Shape of validation data: torch.Size([1900, 6])
Shape of testing data: torch.Size([3800, 6])
global_std.shape: torch.Size([6])
Global Std for lorenz96: tensor([3.6750, 3.6678, 3.7240, 3.7347, 3.8038, 3.5588], device='cuda:0')
Train set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Validation set sample shapes: torch.Size([96, 6]), torch.Size([720, 6])
Test set data shapes: torch.Size([3800, 6]), torch.Size([3800, 6])
Number of batches in train_loader: 98
Batch 0: Data shape torch.Size([128, 96, 6]), Target shape torch.Size([128, 720, 6])

Data Preparation: lorenz96
Sequence Length: 96
Prediction Length: 720
Batch Size: 128
Scaling: No
Train Split: 0.7
Val Split: 0.8
Training Batches: 98
Validation Batches: 9
Test Batches: 24

 Running experiment with seed 1955 (1/3)==================================================

Epoch [1/50], Train Losses: mse: 16.0042, mae: 3.24